# Class 13 Python

- Composition
- Aggregation
- Metaclasses
- Dataclasses

## Diffrence between composition and aggregation

| Feature            | Composition            | Aggregation                                                         |
|--------------------|------------------------|---------------------------------------------------------------------|
| **Definition** | A "has-a" relationship where one object owns another object. | A "has-a" relationship where one object contains another object but does not own it. |
| **Lifespan** | The lifetime of the contained object is tied to the lifetime of the container object. | The contained object can exist independently of the container object. |
| **Dependency** | Strong dependency between the objects. | Weak dependency between the objects. |
| **Example** | A `Car` object contains an `Engine` object, and the engine cannot exist without the car. | A `University` object contains `Departments` objects, but departments can exist without the university. |
| **Implementation** | The contained object is typically created and managed by the container object. | The contained object is passed to the container object, often via parameters. |

## Composition

In [3]:
class Engine:
    def __init__(self, horsepower):
        self.horsepower = horsepower

    def start(self):
        print("Engine started with horsepower:", self.horsepower)


class Car:
    def __init__(self, make, model, horsepower):
        self.make = make
        self.model = model
        self.engine = Engine(horsepower)  # Composition: Car owns an Engine

    def start(self):
        print(f"{self.make} {self.model} is starting...")
        self.engine.start()


# Example usage
my_car = Car("Toyota", "Camry", 200)
my_car.start()
my_car.engine.start()

Toyota Camry is starting...
Engine started with horsepower: 200
Engine started with horsepower: 200


## Aggregation

In [ ]:
class Department:
    def __init__(self, name):
        self.name = name

    def display(self):
        print(f"Department: {self.name}")


class University:
    def __init__(self, name):
        self.name = name
        self.departments: list[Department] = []  # Aggregation: University contains departments

    def add_department(self, department):
        self.departments.append(department)

    def display_departments(self):
        print(f"University: {self.name}")
        for department in self.departments:
            department.display()

    def display(self):
        print(f"University name: {self.name}")


# Example usage
cs_department = Department("Computer Science")
math_department = Department("Mathematics")

university = University("Tech University")
university.add_department(cs_department)
university.add_department(math_department)

university.display_departments()


del university

print('\n========== After deleting university ==========')

math_department.display()
cs_department.display()


University: Tech University
Department: Computer Science
Department: Mathematics

========== After deleting university ==========
Department: Mathematics
Department: Computer Science


## Metaclasses in Python

Metaclasses are the "classes of classes" in Python. They define how classes behave. A class is an instance of a metaclass, just as an object is an instance of a class. By default, Python classes are instances of the `type` metaclass. Metaclasses allow you to customize class creation, such as modifying class attributes or methods during definition. They are defined by inheriting from `type` and overriding methods like `__new__` or `__init__`.


In [ ]:
class MyMeta(type):
    def __new__(cls: type, name: str, bases: tuple[type, ...], attrs: dict[str, object]) -> type:
        # Runs when a class is created
        print(f"MyMeta: Creating class: {name}")

        if "say_hello" not in attrs:
            raise TypeError(f"{name} must define 'say_hello'!")
        
        if not name.startswith('C'):
            raise ValueError("Class name must start with 'C'")


        return super().__new__(cls, name, bases, attrs)
    

class Cat(metaclass=MyMeta):
    def say_hello(self):
        print("Meow!!")



class Dog(metaclass=MyMeta):
    def say_hello(self):
        print("Bark!!")

MyMeta: Creating class: Cat
{'__module__': '__main__', '__qualname__': 'Cat', '__firstlineno__': 17, 'say_hello': <function Cat.say_hello at 0x0000024AE99563E0>, '__static_attributes__': ()}
MyMeta: Creating class: Dog
{'__module__': '__main__', '__qualname__': 'Dog', '__firstlineno__': 23, 'say_hello': <function Dog.say_hello at 0x0000024AE98D9DA0>, '__static_attributes__': ()}


ValueError: Class name must start with 'C'

In [22]:
# Example: Singleton Metaclass (ensures only 1 instance exists)
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    pass

db1: Database = Database()
db2: Database = Database()
print(db1 is db2)  # True (same instance!)

True


## Dataclass in Python?

A dataclass in Python is a decorator introduced in Python 3.7 that simplifies the creation of classes used primarily for storing data. A dataclass automatically adds special methods to the class, like:

- `__init__` : a constructor method to initialize attributes.    
- `__repr__` : a method that provides a human-readable string representation of the object.
- `__eq__` : a method for comparing instances for equality based on their attributes.
- `__hash__` : a method that allows instances of the class to be used as dictionary keys or in sets.

These methods are automatically generated, reducing the need for boilerplate code and making the code more concise and readable.

In [27]:
class Comment():

    def __init__(self, id:int, text: str):
        self.__id = id
        self.__text = text

    @property
    def id(self):
        return self.__id
    
    @property
    def text(self):
        return self.__text

    def __repr__(self):
        return f"Comment(id={self.id}, text={self.text})"
    
    def __eq__(self, value):
        if isinstance(value, Comment):
            return self.id == value.id and self.text == value.text
        return False

    def __ne__(self, value):
        return not self.__eq__(value)

    def __gt__(self, value):
        if isinstance(value, Comment):
            return self.id > value.id
        return NotImplemented

    def __lt__(self, value):
        if isinstance(value, Comment):
            return self.id < value.id
        return NotImplemented

    def __hash__(self):
        return hash((self.id, self.text))


comment = Comment(1, "Hello")
print(comment)
print(comment.id)
print(comment.text)
comment.text = "World"



Comment(id=1, text=Hello)
1
Hello


AttributeError: property 'text' of 'Comment' object has no setter

In [51]:
from dataclasses import dataclass, field
import inspect


@dataclass(frozen=True, order=True, repr=False)
class Comment1():
    id: int
    text: str
    author: str = field(default="Unknown", hash=False)
    replies: list[str] = field(default_factory=list, compare=False, hash=False)


comment = Comment1(1, "Hello")
print(comment)
print(comment.id)
print(comment.text)
print(comment.author)
print(comment.replies)

comment.replies.append("Hi")
print(comment.replies)

# comment.text = "World"

display(inspect.getmembers(Comment1, inspect.isfunction))



1
Hello
Unknown
[]
['Hi']


[('__delattr__', <function __main__.Comment1.__delattr__(self, name)>),
 ('__eq__', <function __main__.Comment1.__eq__(self, other)>),
 ('__ge__', <function __main__.Comment1.__ge__(self, other)>),
 ('__gt__', <function __main__.Comment1.__gt__(self, other)>),
 ('__hash__', <function __main__.Comment1.__hash__(self)>),
 ('__init__',
  <function __main__.Comment1.__init__(self, id: int, text: str, author: str = 'Unknown', replies: list[str] = <factory>) -> None>),
 ('__le__', <function __main__.Comment1.__le__(self, other)>),
 ('__lt__', <function __main__.Comment1.__lt__(self, other)>),
 ('__replace__', <function dataclasses._replace(self, /, **changes)>),
 ('__setattr__', <function __main__.Comment1.__setattr__(self, name, value)>)]